# Pos-taggers comparison for Russian

In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [2]:
import pymorphy2

In [3]:
from nltk import pos_tag, word_tokenize

In [4]:
import re

In [5]:
def preprocess(text):
    return ' '.join(re.findall(r'[а-яё]+', text.lower()))

## NLTK POS-tagger for Russian

In [27]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

__Tagset NLTK__ <br>
S – noun <br>
A – adjective <br> 
NUM – numeral <br>
A-NUM – numeral adjective <br> 
V – verb <br>
ADV – adverb <br>
PRAEDIC — predicative (жаль, хорошо, пора) <br>
PARENTH — parenthesis (кстати, по-моему) <br>
S-PRO — pronoun (она, что) <br>
A-PRO — adjectival pronoun (который, твой) <br>
ADV-PRO — adverbial pronoun (где, вот) <br>
PRAEDIC-PRO — predicative pronoun (некого, нечего) <br>
PR — preposition (под, напротив) <br>
CONJ — conjunction (и, чтобы) <br>
PART — particle (бы, же, пусть) <br>
INTJ — interjection (увы, батюшки)

In [75]:
pos_tag(['прочитавший','прочитав', 'прочитана'], lang = 'rus')

[('прочитавший', 'V'), ('прочитав', 'V'), ('прочитана', 'S')]

#### NLTK tagger function

In [6]:
def get_nltk_tags(text):
    words = preprocess(text).split()
    _, pos = zip(*pos_tag(words, lang = 'rus'))
    nltk_pos = [re.sub('=.+','', pos) for pos in pos]
    return words, nltk_pos

## Pymorphy

In [31]:
morph = pymorphy2.MorphAnalyzer()

__Tagset Pymorphy__ <br>
NOUN	имя существительное	хомяк <br>
ADJF	имя прилагательное (полное)	хороший <br>
ADJS	имя прилагательное (краткое)	хорош <br>
COMP	компаратив	лучше, получше, выше <br>
VERB	глагол (личная форма)	говорю, говорит, говорил <br>
INFN	глагол (инфинитив)	говорить, сказать <br>
PRTF	причастие (полное)	прочитавший, прочитанная <br>
PRTS	причастие (краткое)	прочитана <br>
GRND	деепричастие	прочитав, рассказывая <br>
NUMR	числительное	три, пятьдесят <br>
ADVB	наречие	круто <br>
NPRO	местоимение-существительное	он <br>
PRED	предикатив	некогда <br>
PREP	предлог	в <br>
CONJ	союз	и <br>
PRCL	частица	бы, же, лишь <br>
INTJ	междометие	ой <br>

#### Mapping from nltk tags to Pymorphy

In [7]:
map_nltk_to_pymorphy = {
            'S':'NOUN',
            'A':'ADJF', #краткие прилагательные тоже перейдут в полные, т.к. в nltk прилагательные не разделяются
            'NUM':'NUMR',
            #A-NUM - pymorphy числительное-придагательное идет в прилагательные
            'V':'VERB',
            'ADV':'ADVB',
            'PRAEDIC':'PRED',
            #PARENTH
            'S-PRO':'NPRO',
            #A-PRO pymorphy местоимение-придагательное идет в прилагательные
            #ADV-PRO pymorphy наречие-местоимение идет в местоимение
            #PRAEDIC-PRO pymorphy - идет в местоимение NPRO
            'PR':'PREP',
            #CONJ - одинаково
            'PART':'PRCL'
            #INTJ - одинаково
}

In [71]:
morph.parse('нечего')[0].tag.POS

'NPRO'

In [8]:
def nltk_to_pymorphy(tag_list):
    pymorphy_tags = []
    for tag in tag_list:
        try:
            tag = map_nltk_to_pymorphy[tag]
        except KeyError:
            pass
        pymorphy_tags.append(tag)
    return pymorphy_tags
            

        

#### Pymorphy tagger function

In [10]:
def get_pymorphy_tags(text):
    morph = pymorphy2.MorphAnalyzer()
    words = preprocess(text).split()
    return words, [morph.parse(word)[0].tag.POS for word in words]
    

### Pymorphy vs. NLTK tagger

1. В Pymorphy отдельные классы для глагольных форм (финитные глаголы, инфинитивы, причастия, краткие причастия, деепричастия). В NLTK - глагольные формы (за исключением некоторых причастий - страдательных?) относятся к классу глаголов.
2. В Pymorphy разделяются краткие и полные прилагательные.
3. В NLTK выделяются классы для слов типа прилагательных: порядковые числительные, местоимения-прилагательные, отдельно выделяются классы для разных местоимений: местоимения-прилагательные, местоимения-наречия, предикативные местоимения. 

## Mystem 

In [11]:
from pymystem3 import Mystem

__Tagset Mystem__ <br>
A	прилагательное <br>
ADV	наречие <br>
ADVPRO	местоименное наречие <br>
ANUM	числительное-прилагательное <br>
APRO	местоимение-прилагательное <br>
COM	часть композита - сложного слова <br>
CONJ	союз <br>
INTJ	междометие <br>
NUM	числительное <br>
PART	частица <br>
PR	предлог <br>
S	существительное <br>
SPRO	местоимение-существительное <br>
V	глагол <br>

In [96]:
m = Mystem()

#### Mystem tagger function

In [12]:
def get_mystem_tags(text):
    m = Mystem()
    clean_text = preprocess(text)
    analized = m.analyze(clean_text)
    mystem_tags = []
    for i in range(len(analized)):

        try:
            pos_t = analized[i]['analysis'][0]['gr']
            pos_t = re.sub(',.*','', pos_t)
            pos_t = re.sub('=.*','',pos_t)
            mystem_tags.append(pos_t)
        except KeyError:
            pass
    return clean_text.split(), mystem_tags

### Mystem vs. NLTK

Mystem has new tag compared to NLTK - COM (part of a compound noun), and NLTK has more tags specifying predicatives and predicative pronouns. <br>
Since Mystem and NLTK share most of tags, we'll map Mystem to NLTK and then NLTK to Pymorphy. <br>


#### Mapping Mystem tags to NLTK

In [13]:
map_mystem_to_nltk = {
    'ADVPRO':'ADV-PRO',
    'ANUM':'A-NUM',
    'APRO':'A-PRO',
    'SPRO':'S-PRO'
}

#### General function for mapping tags from input (in_tags) to output using tag_map

In [14]:
def mapping_tags(in_tags, tag_map):
    out_tags = []
    for tag in in_tags:
        try:
            tag = tag_map[tag]
        except KeyError:
            pass
        out_tags.append(tag)
    return out_tags

In [135]:
print(mystem_tags)
print(mapping_tags(mystem_tags, map_mystem_to_nltk))
print(nltk_pos)
print(mapping_tags(nltk_pos, map_nltk_to_pymorphy))
print(mapping_tags(mapping_tags(mystem_tags, map_mystem_to_nltk), map_nltk_to_pymorphy))
print(pymorphy_tags)

['S', 'SPRO', 'V', 'APRO', 'S', 'SPRO', 'V', 'S']
['S', 'S-PRO', 'V', 'A-PRO', 'S', 'S-PRO', 'V', 'S']
['S', 'S-PRO', 'V', 'A-PRO', 'S', 'S-PRO', 'V', 'S']
['NOUN', 'NPRO', 'VERB', 'A-PRO', 'NOUN', 'NPRO', 'VERB', 'NOUN']
['NOUN', 'NPRO', 'VERB', 'A-PRO', 'NOUN', 'NPRO', 'VERB', 'NOUN']
['NOUN', 'NPRO', 'VERB', 'ADJF', 'NOUN', 'NPRO', 'VERB', 'NOUN']


### Test sentence

In [15]:
text2 = 'Уколов меня, врач вышел из комнаты. Я не боюсь уколов'

In [72]:
def compare_taggers(text):
    words, pymorph_tags = get_pymorphy_tags(text)
    _, nltk_tags = get_nltk_tags(text)
    _, mystem_tags = get_mystem_tags(text)
    return words, pymorph_tags, mapping_tags(nltk_tags, map_nltk_to_pymorphy), \
            mapping_tags(mapping_tags(mystem_tags, map_mystem_to_nltk), map_nltk_to_pymorphy)

In [77]:
w, p, n, m = compare_taggers(text2)

In [78]:
print(w)
print(p)
print(n)
print(m)

['уколов', 'меня', 'врач', 'вышел', 'из', 'комнаты', 'я', 'не', 'боюсь', 'уколов']
['NOUN', 'NPRO', 'NOUN', 'VERB', 'PREP', 'NOUN', 'NPRO', 'PRCL', 'VERB', 'NOUN']
['NOUN', 'NPRO', 'NOUN', 'VERB', 'PREP', 'NOUN', 'NPRO', 'PRCL', 'VERB', 'NOUN']
['NOUN', 'NPRO', 'NOUN', 'VERB', 'PREP', 'NOUN', 'NPRO', 'PRCL', 'VERB', 'NOUN']


This sentence shows that all the pos-taggers failed to disambiguate participle __уколов__ and tagged it as noun.

In [79]:
text3 = 'Он стоял на берегу, всматриваясь в синие дали'

In [81]:
w, p, n, m = compare_taggers(text3)
print(w)
print(p)
print(n)
print(m)

['он', 'стоял', 'на', 'берегу', 'всматриваясь', 'в', 'синие', 'дали']
['NPRO', 'ADJS', 'PREP', 'NOUN', 'GRND', 'PREP', 'ADJF', 'VERB']
['NPRO', 'VERB', 'PREP', 'NOUN', 'VERB', 'PREP', 'NOUN', 'VERB']
['NPRO', 'VERB', 'PREP', 'NOUN', 'VERB', 'PREP', 'ADJF', 'NOUN']


Here the word __дали__ is the most interesting because is could present both verb and noun, though verb is much more common. Only Mystem tagger managed to disambiguate it correctly as a noun. <br>
There are some more mismatches:<br>
Pymorphy gives label ADJS (short adjective) for verb 'стоял', and that's the only tagger that determines gerund 'всматриваясь' (participle). <br>
NLTK tagger states NOUN for adjective 'синие'.

In [82]:
text4 = 'Нам всем дали высказаться'

In [83]:
w, p, n, m = compare_taggers(text4)
print(w)
print(p)
print(n)
print(m)

['нам', 'всем', 'дали', 'высказаться']
['NPRO', 'ADJF', 'VERB', 'INFN']
['NPRO', 'NPRO', 'VERB', 'VERB']
['NPRO', 'A-PRO', 'VERB', 'VERB']


In this case all the taggers determined word __дали__ correctly as a verb.

### Test text

Let's compare taggers on a small text by R. Bradbury

In [16]:
import pandas as pd

In [69]:
pos_df = pd.DataFrame(columns = ['word', 'pymorph', 'nltk', 'mystem'])

In [71]:
text = 'Одно прикосновение руки - и тотчас это горение послушно даст задний ход. Экельс помнил каждое слово объявления. Из пепла и праха, из пыли и золы восстанут, будто золотистые саламандры, старые годы, зеленые годы, розы усладят воздух, седые волосы станут черными, исчезнут морщины и складки, все и вся повернет вспять и станет семенем, от смерти ринется к своему истоку, солнца будут всходить на западе и погружаться в зарево востока, луны будут убывать с другого конца, все и вся уподобится цыпленку, прячущемуся в яйцо, кроликам, ныряющим в шляпу фокусника, все и вся познает новую смерть, смерть семени, зеленую смерть, возвращения в пору, предшествующую зачатию. И это будет сделано одним лишь движением руки...'

In [73]:
w, p, n, m = compare_taggers(text)

In [74]:
pos_df['word'] = w
pos_df['pymorph'] = p
pos_df['nltk'] = n
pos_df['mystem'] = m

In [75]:
pos_df.shape

(109, 4)

In [76]:
pos_df

,word,pymorph,nltk,mystem
0,одно,ADJF,A-PRO,A-PRO
1,прикосновение,NOUN,NOUN,NOUN
2,руки,NOUN,NOUN,NOUN
3,и,CONJ,CONJ,CONJ
4,тотчас,ADVB,ADVB,ADVB
5,это,PRCL,A-PRO,A-PRO
6,горение,NOUN,NOUN,NOUN
7,послушно,ADVB,ADVB,ADVB
8,даст,VERB,VERB,VERB
9,задний,ADJF,ADJF,ADJF


Again mismatch in tags for most cases is related to different asignment of tags for adjective-type words (in Pymorphy all of them are labeled as adjective, in Mystem and NLTK adgective-number and adjective-pronoun are specified). <br>
The same thing happens to verb forms. <br>


An interesting diambiguation case could be noticed for word __это__. 

In [95]:
pos_df[5:9]

,word,pymorph,nltk,mystem
5,это,PRCL,A-PRO,A-PRO
6,горение,NOUN,NOUN,NOUN
7,послушно,ADVB,ADVB,ADVB
8,даст,VERB,VERB,VERB


Here 'это' is a determinative pronoun for noun 'горение'. NLTK and Mystem gave correct tag -- adjective-pronoun while Pymorphy labeled it as a particle.

In [97]:
pos_df[101:105]

,word,pymorph,nltk,mystem
101,и,CONJ,CONJ,CONJ
102,это,PRCL,NPRO,NPRO
103,будет,VERB,VERB,VERB
104,сделано,PRTS,VERB,VERB


In this case 'это' is not related to a noun so NLTK and Mystem correctly label it as a noun-pronoun. 

### Conclusion

Mostly NLTK, Mystem and Pymorphy give pretty good ressults for POS-tagging, but it seems like NLTK and Mystem have more power for morphological disambiguation and Mystem is slower than Pymorphy and NLTK. <br>
Different tagsets also give different possibilities for POS-tagging.